## Notebook for the Viszualization of reconstructed Variance with reduced model parameters

### ToDo:
    - Documentation
    - fix paths

In [ ]:
import errno  # handy system and path functions
import glob
import locale
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import os

import pandas as pd
import sys  # to get file system encoding
import scipy.linalg
from scipy import signal
from scipy import stats
import seaborn as sns
from sklearn import preprocessing


# set Matplotlib parameters globally
params = {'legend.fontsize': 14,
          'legend.handlelength': 2,
         'figure.autolayout': True,
         'font.serif' : 'Ubuntu',
         'font.family' : 'serif',
         'font.monospace' : 'Ubuntu Mono',
         'font.size' : 12,
         'axes.labelsize' : 16,
         'axes.titlesize' : 16,
         'xtick.labelsize' : 14,
         'ytick.labelsize' : 14,
         'figure.titlesize' : 12,
         'figure.dpi' : 300}

rcParams.update(params)


# set Seaborn parameters globally
sns.set_context("paper")
sns.set_style('whitegrid')
sns.set_palette('colorblind')

In [ ]:
%run ba_analysis_1h_functions.py

In [ ]:
marker_dict.values()

In [ ]:
def get_model(dimensions, event_file, data_file):
    '''function to extract and resample movement events,
    calculate mean and variance of trahectories for each kinematic dimension
    and zero-center trajectories for further use in a SVD
    
    input: 
        dimensions = dimensionality of a model; One dimension consists of 1 kinematic dimension + one cartesian dimension
                    e.g. full scale model for right arm:
                    dimensions = right_hand_marker[:1] + right_wrist + right_lower_arm  + right_upper_arm + right_shoulder 
        event_file = dataframe containing information on which trials to include
        data_file = Dataframe containing all recorded movement data
        
    output:
        model containing zero-centered trajectories for specified dimensions
    '''
    df_model_a = pd.DataFrame()

    for i in dimensions:
        i = str(i)

        marker = ['marker_'+i+'_x', 'marker_'+i+'_y', 'marker_'+i+'_z']

        df_x, df_y, df_z = extract_and_resample(event_file, data_file, marker)
        x_mean_dimension, x_variance_dimension, x_df_corrected = get_corrected('x', df_x)
        y_mean_dimension, y_variance_dimension, y_df_corrected = get_corrected('y', df_y)
        z_mean_dimension, z_variance_dimension, z_df_corrected = get_corrected('z', df_z)

        reshaped_df = reshape(x_df_corrected, y_df_corrected, z_df_corrected)

        for idx, j in enumerate(reshaped_df.columns):
            df_model_a[str(marker[idx])] = reshaped_df[j]
    return df_model_a

In [ ]:
def specify_model(model, parameters):
    '''
    Function to further specify which dimensions of full scale model created by get_model function 
    to use for further analysis.
    
    input:
        model = model containing zero-centered trajectories for dimensions of interest
        parameters = list of ones and zeros indicating which kinnematich degree of freedom to extract
        
    output:
        list of parameters by which Dataframe containing full scale model can be truncated.
    '''
    model_parameters = []
    for idx, i in enumerate(parameters):  
        if i == 1:  # if parameter should be included it is encoded with a 1
            model_parameters.append(model.columns[idx])
    return model_parameters

In [ ]:
def modelComparison(data):
    """Decompose data with SVD and compute BIC-like approximation to log(P(data|number of primitives))
    for each possible number of primitives
    returns: variance-accounted-for and log P for each number of synergies"""
    
    U,S,VT=np.linalg.svd(data)
    
    SVT=np.diag(S).dot(VT)
    
    tot_var=data.var() # this may have to be computed per trial, i.e. as data.var(axis=1).mean()
    N=data.size
    
    VAF=[]
    logPD=[]
    
    for num_prim in range(len(SVT)-2): # -2 to avoid 
        
        sub_u=U[:,:num_prim+1]
        sub_svt=SVT[:num_prim+1]
        reconstruction=np.dot(sub_u,sub_svt)
        
        tot_num_df=sub_u.size+sub_svt.size # total number of degrees of freedom in the model
        
        rec_err_2= ((data-reconstruction)**2).mean()
        VAF.append(1.0-rec_err_2/tot_var)
        log_likelihood=-N/2*(np.log(2*np.pi)+np.log(rec_err_2)+1)
        logPD.append(log_likelihood-0.5*tot_num_df*np.log(N))
        fig = plt.figure(figsize=(16,9))
        plt.plot(reshaped['x'], color='blue', alpha=0.4)
        plt.plot(reconstruction[:,0], color='red')
        plt.show()
        
    return VAF,np.array(logPD), reconstruction

In [ ]:
def plotting(marker_, pos, pos_name, subject_id, path, parameters, data_file, reconstruct_number):
 
    # create subj directory
    directory = path+'/graphs/1H/'+subject_id+'/'+pos_name
    try:
        os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    # create directory for ouputs on  velocity
    try:
        os.makedirs(directory+'/velocity/')
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    # create directory for ouputs on  trajectories
    try:
        os.makedirs(directory+'/trajectories/')
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    
    i = marker_ #  marker of specified hand
    # define Marker as dictionary with entry for marker in each cartesian dimension
    marker_ =['marker_'+i+'_x', 'marker_'+i+'_y', 'marker_'+i+'_z']
    

    # extract specifics of target postion
    box_pos_x = -1*(pos['box_pos_x'][0])
    box_pos_z_r = pos['box_pos_z'][0]
    box_pos_y_r = pos['box_pos_y'][0]
    print('!!!!! Traget Pos: z = '+ str(box_pos_z_r)+ ' ; y = '+ str(box_pos_y_r) )
    
    # Ectract and resample movement date for trials with specififed target position for specified marker
    df_x_r, df_y_r, df_z_r = extract_and_resample(pos, data_file, marker_)
    # extract mean trajectory, variance, and zero-centered trajectory for each cartesian dimension
    x_mean_dimension_r, x_variance_dimension_r, x_df_corrected_r = get_corrected('x', df_x_r)
    y_mean_dimension_r, y_variance_dimension_r, y_df_corrected_r = get_corrected('y', df_y_r)
    z_mean_dimension_r, z_variance_dimension_r, z_df_corrected_r = get_corrected('z', df_z_r)

    # output extracted data as csvs
    df_trajectories = pd.DataFrame()
    df_trajectories['x_dim_'+subject_id+'_'+pos_name] = x_mean_dimension_r
    df_trajectories['y_dim_'+subject_id+'_'+pos_name] = y_mean_dimension_r
    df_trajectories['z_dim_'+subject_id+'_'+pos_name] = z_mean_dimension_r
    df_trajectories['x_dim_var_'+subject_id+'_'+pos_name] = x_variance_dimension_r
    df_trajectories['y_dim_var_'+subject_id+'_'+pos_name] = y_variance_dimension_r
    df_trajectories['z_dim_var_'+subject_id+'_'+pos_name] = z_variance_dimension_r
    df_trajectories['box_pos_x'] = box_pos_x
    df_trajectories['box_pos_z'] = box_pos_z_r
    df_trajectories['box_pos_y'] = box_pos_y_r
    df_trajectories.to_csv(path+'/graphs/1H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_trajectories.csv', index=False, encoding='utf-8')
    reshaped_df_r = reshape(x_df_corrected_r, y_df_corrected_r, z_df_corrected_r)
    reshaped_df_r.to_csv(path+'/graphs/1H/'+subject_id+'/'+pos_name+'/trajectories/'+pos_name+'_corrected_trajectories.csv', index=False, encoding='utf-8')

    # Transform data into np-arrays for easier plotting
    x = np.array(x_mean_dimension_r)
    z=  np.array(z_mean_dimension_r)
    y = np.array(y_mean_dimension_r)
    
    # lower boundaries of variance
    x_l = x - np.array(x_variance_dimension_r)
    z_l = z - np.array(z_variance_dimension_r)
    y_l  = y - np.array(y_variance_dimension_r)

    # upper boundaries of variance
    x_u = x + np.array(x_variance_dimension_r)
    z_u = z + np.array(z_variance_dimension_r)
    y_u  = y + np.array(y_variance_dimension_r)

    
    ### Plot first mean loading on first vector of U-Matrix
    # define dimensionality in marker space    
    if subject_id == 'sub-05':
        # left_hand_marker = [4,5,6] -> specified in ba_analysis_1h_functions.pyl
        dimensions = left_hand_marker[:1] + left_wrist + left_lower_arm  + left_upper_arm + left_shoulder 
    else:
        dimensions = right_hand_marker[:1] + right_wrist + right_lower_arm  + right_upper_arm + right_shoulder 
#   

    # specify Model for dominant arm only
    model_r = get_model(dimensions, pos, data_file)
    # specify which parameters to extract
    model_parameters = specify_model(model_r, parameters)
    # build model
    model_r = model_r[model_parameters]
    
#####################################################################################################
    
    data = np.array(model_r)
    U,S,VT=scipy.linalg.svd(data)
    
    SVT=np.diag(S).dot(VT)
    
    tot_var=data.var() # this may have to be computed per trial, i.e. as data.var(axis=1).mean()
    N=data.size
    
    VAF=[]
    logPD=[]

    for num_prim in range(len(SVT)-2): # -2 to avoid 
        
        sub_u=U[:,:reconstruct_number+1]
        sub_svt=SVT[:reconstruct_number+1]
        reconstruction=np.dot(sub_u,sub_svt)
#         df_reconstructed['pc_'+str(counter)] = reconstruction
        tot_num_df=sub_u.size+sub_svt.size # total number of degrees of freedom in the model

#         fig = plt.figure(figsize=(14,7))
#         plt.plot(reconstruction)
#         plt.plot(reshaped['x'], color='blue', alpha=0.4)
#         plt.show()
        
        
        rec_err_2= ((data-reconstruction)**2).mean()
        VAF.append(1.0-rec_err_2/tot_var)
        log_likelihood=-N/2*(np.log(2*np.pi)+np.log(rec_err_2)+1)
        logPD.append(log_likelihood-0.5*tot_num_df*np.log(N))
        
    display(model_r.columns)

    return model_r,  x_df_corrected_r,  y_df_corrected_r,  z_df_corrected_r, reconstruction
 

In [ ]:
dir_ = '/home/michael/bachelorarbeit/subjects/*'
# dir_ = '/home/michael/bachelorarbeit/subjects/sub-09/'

path = '/home/michael/bachelorarbeit'
list_trajectories_r_x = []
list_trajectories_r_y = []
list_trajectories_r_z = []
list_mean_loadings = []
df_velocities = pd.DataFrame()
df_mean_loadings = pd.DataFrame()
parameters = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
palette = sns.color_palette("BuGn", 9)
palette_original = sns.color_palette("OrRd", 10)
subjects = []
df_reconstructions = pd.DataFrame()
df_originals = pd.DataFrame()
marker = '7'
# marker = '4'

df_r1 = pd.DataFrame()
subjects_info = pd.DataFrame()
# for number_of_primitives in range(13):
# reconstruct_number = number_of_primitives+1
reconstruct_number = 4
for subject in (glob.glob(dir_)):
    subject_id = subject.split('/')[5]
    print(subject_id)
    subjects.append(subject_id)
    if subject_id == 'sub-05':  # left-handed
        marker = '5'
    else:
        marker = '7'
    for filename in (glob.glob(subject+'/*')):
        if filename.split('_')[2] == 'parameters':
            subject_parameters = pd.read_csv(filename)
            subjects_info = subjects_info.append(subject_parameters)
        elif filename.split('-')[3] == '1H' and filename.split('-')[4] == 'events':
            event_file_1H = pd.read_csv(filename)
        elif filename.split('-')[3] == '1H' and filename.split('-')[4] == 'data':
            data_file_1H = pd.read_csv(filename)
    r1,r2,r3,m1,m2,m3,l1,l2,l3 = group_by_position(event_file_1H)
    positions_list = [r1,r2,r3,m1,m2,m3,l1,l2,l3]
    pos_names = ['r1','r2','r3','m1','m2','m3','l1','l2','l3']

    for idx, i in enumerate(positions_list):
        df_reconstructed = pd.DataFrame()
        df_original = pd.DataFrame()
        reshaped, df_x, df_y, df_z, reconstruction = plotting(marker, i, pos_names[idx],
                                              subject_id, path,parameters,  data_file_1H,
                                             reconstruct_number)

        for j in range(len(reshaped.columns)):
            # RESHAPE ORIGINAL MATRIX
            counter = 0
            trial_length = 200
            concatenated = pd.DataFrame(columns=range(len(reshaped)//trial_length))
            reshaped = np.array(reshaped)
            for i in range(len(reshaped)//trial_length):

                counter +=trial_length
                if i == 0:
                    concatenated[concatenated.columns[i]] = reshaped[:counter,j]
                elif i >0:
                    concatenated[concatenated.columns[i]] = reshaped[counter-trial_length:counter,j]

            concatenated = concatenated.values
            concatenated = np.absolute(concatenated)

            # extract mean loading and standarad error of mean
            mean_original = []
            stdm_original = []
            for i in range(trial_length):
                mean_original.append(concatenated[i].mean())
                stdm_original.append(scipy.stats.sem(concatenated[i]))

            mean_original = np.array(mean_original)
            df_original[pos_names[idx]+'_'+str(j)] = mean_original

            stdm_original = np.array(stdm_original)


            # RESHAPE RECONSTRUCTED MATRIX
            counter = 0
            trial_length = 200
            recon_ = pd.DataFrame(columns=range(len(reconstruction)//trial_length))
            for i in range(len(reconstruction)//trial_length):

                counter +=trial_length
                if i == 0:
                    recon_[recon_.columns[i]] = reconstruction[:counter,j]
                elif i >0:
                    recon_[recon_.columns[i]] = reconstruction[counter-trial_length:counter,j]

            # transform trial to matrix and transform each point to absolute value            
            recon_ = recon_.values
            recon_ = np.absolute(recon_)

            # extract mean loading and standarad error of mean
            mean_recon = []
            stdm_recon = []
            for i in range(trial_length):
                mean_recon.append(recon_[i].mean())
                stdm_recon.append(scipy.stats.sem(recon_[i]))

            mean_recon = np.array(mean_recon)
            df_reconstructed[pos_names[idx]+'_'+str(j)] = mean_recon
            stdm_recon = np.array(stdm_recon)

        counter_marker_pos = 0
        fig = plt.figure(figsize=(15,8))
        ax = fig.add_subplot(111, projection='3d')
        for x in range(len(df_reconstructed.columns)):

            if counter_marker_pos > 12:
                print('done')
            elif counter_marker_pos <= 12:

                if counter_marker_pos == 0:
                    kin_dim = 'hand'
                    colour='navy'
                    colour_orig='navy'

                elif counter_marker_pos == 3:
                    kin_dim = 'wrist'
                    colour= 'forestgreen'
                    colour_orig='forestgreen'
                elif counter_marker_pos == 6:
                    kin_dim = 'lower arm'
                    colour='darkgoldenrod'
                    colour_orig='darkgoldenrod'
                elif counter_marker_pos == 9:
                    kin_dim = 'upper arm'
                    colour='maroon'
                    colour_orig='maroon'
                elif counter_marker_pos == 12:
                    kin_dim = 'shoulder'
                    colour='black'
                    colour_orig='black'

                x = df_reconstructed[df_reconstructed.columns[counter_marker_pos]]
                y = df_reconstructed[df_reconstructed.columns[counter_marker_pos+1]]
                z = df_reconstructed[df_reconstructed.columns[counter_marker_pos+2]]

                x_orig = df_original[df_original.columns[counter_marker_pos]]
                y_orig = df_original[df_original.columns[counter_marker_pos+1]]
                z_orig = df_original[df_original.columns[counter_marker_pos+2]]

                ### PLOT TRAJECTTORIES IN 3D
                # right hand
                ax.plot(xs=x, ys=z, zs=y,color=colour, label=(kin_dim+'_reconstructed'))
                ax.plot(xs=x_orig, ys=z_orig, zs=y_orig, color=colour_orig, label=(kin_dim+'_original'), linestyle=':')

                counter_marker_pos += 3
        ax.set_xlim(0,.2)
        ax.set_zlim(0,.2)
        ax.set_ylim(0,.2)
        ax.view_init(azim=235)

        ax.set_xlabel('x', fontsize=18)
        ax.set_ylabel('z', fontsize=18)
        ax.set_zlabel('y', fontsize=18)

        plt.legend(bbox_to_anchor=(0.06, 1.1),
          fancybox=True, shadow=True,  borderaxespad=0.)
        plt.title(subject_id+' '+pos_names[idx]+': number of principal components '+str(reconstruct_number), y=-.05)
        fig.savefig(path+'/graphs/1H/'+subject_id+'/'+subject_id+'_'+pos_names[idx]+'_pcs_'+str(reconstruct_number)+'_.png')

        plt.show()

        df_reconstructions[subject_id+' '+pos_names[idx]+': number of principal components '+str(reconstruct_number)] = mean_recon
        df_originals[subject_id+' '+pos_names[idx]+': number of principal components '+str(reconstruct_number)] = mean_original